In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
# LU look up file for health codes
## refer to "inpatient_data_preprocessing_.ipynb"
LU_ICD = pd.read_csv('/content/drive/MyDrive/Data/icd10/ICD10Diagnosis.csv')
LU_ICD.rename(columns={'Code': 'ICD_DIAG_CD', 'Description': 'ICD_Description'}, inplace=True)

outpatient = pd.read_csv('/content/drive/MyDrive/Data/Input/outpatient.csv', sep='|')
col_outpatient = pd.DataFrame(outpatient.dtypes, columns=['type'])
#col_outpatient

<ipython-input-29-dada294be7ed>:6: DtypeWarning: Columns (9,55,56,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  outpatient = pd.read_csv('/content/drive/MyDrive/Data/Input/outpatient.csv', sep='|')


In [30]:
# convert columns to appropriate data types
outpatient['BENE_ID'] = outpatient['BENE_ID'].astype(str)
outpatient['CLM_ID'] = outpatient['CLM_ID'].astype(str)
outpatient['PTNT_DSCHRG_STUS_CD'] = outpatient['PTNT_DSCHRG_STUS_CD'].astype(str)

# dates columns
outpatient['CLM_FROM_DT'] = pd.to_datetime(outpatient['CLM_FROM_DT'], format='%d-%b-%Y')
outpatient['CLM_THRU_DT'] = pd.to_datetime(outpatient['CLM_THRU_DT'], format='%d-%b-%Y')
outpatient['YR'] = outpatient['CLM_FROM_DT'].dt.year

# drop and select columns
columns_to_drop = [col for col in outpatient.columns if col.endswith('UPIN') or col.startswith('ICD_DGNS_E_CD') or col.startswith('PRCDR_DT')]
outpatient = outpatient.drop(columns=columns_to_drop)
outpatient

,BENE_ID,CLM_ID,NCH_NEAR_LINE_REC_IDENT_CD,NCH_CLM_TYPE_CD,CLM_FROM_DT,CLM_THRU_DT,NCH_WKLY_PROC_DT,FI_CLM_PROC_DT,CLAIM_QUERY_CODE,PRVDR_NUM,CLM_FAC_TYPE_CD,CLM_SRVC_CLSFCTN_TYPE_CD,CLM_FREQ_CD,FI_NUM,CLM_MDCR_NON_PMT_RSN_CD,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,NCH_PRMRY_PYR_CD,PRVDR_STATE_CD,ORG_NPI_NUM,AT_PHYSN_NPI,OP_PHYSN_NPI,OT_PHYSN_NPI,CLM_MCO_PD_SW,PTNT_DSCHRG_STUS_CD,CLM_TOT_CHRG_AMT,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,NCH_PROFNL_CMPNT_CHRG_AMT,PRNCPAL_DGNS_CD,ICD_DGNS_CD1,ICD_DGNS_CD2,ICD_DGNS_CD3,ICD_DGNS_CD4,ICD_DGNS_CD5,ICD_DGNS_CD6,ICD_DGNS_CD7,ICD_DGNS_CD8,ICD_DGNS_CD9,ICD_DGNS_CD10,ICD_DGNS_CD11,ICD_DGNS_CD12,ICD_DGNS_CD13,ICD_DGNS_CD14,ICD_DGNS_CD15,ICD_DGNS_CD16,ICD_DGNS_CD17,ICD_DGNS_CD18,ICD_DGNS_CD19,ICD_DGNS_CD20,ICD_DGNS_CD21,ICD_DGNS_CD22,ICD_DGNS_CD23,ICD_DGNS_CD24,ICD_DGNS_CD25,FST_DGNS_E_CD,ICD_PRCDR_CD1,ICD_PRCDR_CD2,ICD_PRCDR_CD3,ICD_PRCDR_CD4,ICD_PRCDR_CD5,ICD_PRCDR_CD6,ICD_PRCDR_CD7,ICD_PRCDR_CD8,ICD_PRCDR_CD9,ICD_PRCDR_CD10,ICD_PRCDR_CD11,ICD_PRCDR_CD12,ICD_PRCDR_CD13,ICD_PRCDR_CD14,ICD_PRCDR_CD15,ICD_PRCDR_CD16,ICD_PRCDR_CD17,ICD_PRCDR_CD18,ICD_PRCDR_CD19,ICD_PRCDR_CD20,ICD_PRCDR_CD21,ICD_PRCDR_CD22,ICD_PRCDR_CD23,ICD_PRCDR_CD24,ICD_PRCDR_CD25,RSN_VISIT_CD1,RSN_VISIT_CD2,RSN_VISIT_CD3,NCH_BENE_PTB_DDCTBL_AMT,NCH_BENE_PTB_COINSRNC_AMT,CLM_OP_PRVDR_PMT_AMT,CLM_OP_BENE_PMT_AMT,CLM_LINE_NUM,REV_CNTR,REV_CNTR_DT,REV_CNTR_1ST_ANSI_CD,REV_CNTR_2ND_ANSI_CD,REV_CNTR_3RD_ANSI_CD,REV_CNTR_4TH_ANSI_CD,REV_CNTR_APC_HIPPS_CD,HCPCS_CD,HCPCS_1ST_MDFR_CD,HCPCS_2ND_MDFR_CD,REV_CNTR_PMT_MTHD_IND_CD,REV_CNTR_DSCNT_IND_CD,REV_CNTR_PACKG_IND_CD,REV_CNTR_OTAF_PMT_CD,REV_CNTR_IDE_NDC_UPC_NUM,REV_CNTR_UNIT_CNT,REV_CNTR_RATE_AMT,REV_CNTR_BLOOD_DDCTBL_AMT,REV_CNTR_CASH_DDCTBL_AMT,REV_CNTR_COINSRNC_WGE_ADJSTD_C,REV_CNTR_RDCD_COINSRNC_AMT,REV_CNTR_1ST_MSP_PD_AMT,REV_CNTR_2ND_MSP_PD_AMT,REV_CNTR_PRVDR_PMT_AMT,REV_CNTR_BENE_PMT_AMT,REV_CNTR_PTNT_RSPNSBLTY_PMT,REV_CNTR_PMT_AMT_AMT,REV_CNTR_TOT_CHRG_AMT,REV_CNTR_NCVRD_CHRG_AMT,REV_CNTR_STUS_IND_CD,REV_CNTR_NDC_QTY,REV_CNTR_NDC_QTY_QLFR_CD,RNDRNG_PHYSN_NPI,YR
0,-10000010254618,-10000930037832,W,40,2015-03-16,2015-03-16,20-Mar-2015,NaN,3,01S023,1,3,1,NaN,,10224.89,0.00,,1,1942394739,9999995696,9999995696,NaN,0,1,10224.89,0,4,R4689,R4689,E781,J329,E119,D649,E849,B965,E848,E849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,20,10224.89,0,1,1,16-Mar-2015,NaN,NaN,NaN,NaN,NaN,99241,NaN,NaN,4,NaN,NaN,NaN,NaN,1,NaN,0,0.00,2044.97,2044.97,0,0,10224.89,0,2044.97,10224.89,8179.92,0.00,4,NaN,NaN,9999995696,2015
1,-10000010254618,-10000930037833,W,40,2015-04-27,2015-04-27,01-May-2015,NaN,3,01S023,1,3,1,NaN,,11936.39,0.00,,1,1942394739,9999995696,9999995696,NaN,0,1,11936.39,0,4,R4689,S134XX,Z7289,E781,J329,E119,D649,E849,B965,E848,E849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,20,11936.39,0,1,1,27-Apr-2015,NaN,NaN,NaN,NaN,NaN,99241,NaN,NaN,4,NaN,NaN,NaN,NaN,1,NaN,0,0.00,2387.27,2387.27,0,0,11936.39,0,2387.27,11936.39,9549.12,0.00,4,NaN,NaN,9999995696,2015
2,-10000010254618,-10000930037834,W,40,2015-05-23,2015-05-23,29-May-2015,NaN,3,01S023,1,3,1,NaN,,8674.15,0.00,,1,1942394739,9999995696,9999995696,NaN,0,1,8674.15,0,4,R4689,S134XX,Z7289,E781,J329,E119,D649,E089,B965,E848,E849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,8674.15,0,1,1,23-May-2015,NaN,NaN,NaN,NaN,NaN,99241,NaN,NaN,4,NaN,NaN,NaN,NaN,1,NaN,0,0.00,1734.83,1734.83,0,0,8674.15,0,1734.83,8674.15,6939.32,0.00,4,NaN,NaN,9999995696,2015
3,-10000010254618,-10000930037835,W,40,2015-06-01,2015-06-01,05-Jun-2015,NaN,3,01S023,1,3,1,NaN,,1171.57,0.00,,1,1942394739,9999995696,9999995696,NaN,0,1,1171.57,0,4,R4689,R4689,E781,J329,E119,D649,E

In [31]:
outpatient['REV_CNTR'].value_counts()

,count
REV_CNTR,
1,574480
780,317
789,295


* `rev_center` (revenue center) of 450 means ER
* No ER claims exist in this data (but expect lots of 450s in the inpatient file, like we saw based on `ER_flag` in "inpatient_data_preprocessing_.ipynb")
* `rev_center` of 780 and 789 ralates with telemedicine, so we can ignore...

In [33]:
outpatient = outpatient[outpatient['CLM_LINE_NUM'] == 1]

#compute LOS (NB: usually 1 for all outpatient data, but could be more than 1 day for patients under observation/recovery)
outpatient['LOS'] = (outpatient['CLM_THRU_DT'] - outpatient['CLM_FROM_DT']).dt.days + 1
outpatient['LOS'].value_counts()

,count
LOS,
1,371460
2,31087
3,81
8,22
4,2
48,1


In [34]:
diagnosis = outpatient[['BENE_ID', 'YR', 'PRNCPAL_DGNS_CD'] + [col for col in outpatient.columns if col.startswith('ICD_DGNS_CD')]]

# wide to long format data, and remove rows where ICD_DIAG_CD is not populated
diagnosis = pd.melt(diagnosis, id_vars=['BENE_ID', 'YR'], var_name='ICD_DIAG_COL', value_name='ICD_DIAG_CD')
diagnosis = diagnosis[diagnosis['ICD_DIAG_CD'].notna() & (diagnosis['ICD_DIAG_CD'] != '')]

# drop the 'ICD_DIAG_COL', then remove duplicates
diagnosis = diagnosis.drop(columns=['ICD_DIAG_COL'])
diagnosis = diagnosis.drop_duplicates()

diagnosis[diagnosis['BENE_ID'] == '-10000010254618'] #randomly viewing a beneficiary

,BENE_ID,YR,ICD_DIAG_CD
0,-10000010254618,2015,R4689
14,-10000010254618,2016,R4689
22,-10000010254618,2016,B9789
33,-10000010254618,2017,B9789
48,-10000010254618,2018,B9789
...,...,...,...
4831900,-10000010254618,2021,E848
4831908,-10000010254618,2022,E848
4831909,-10000010254618,2023,N469
5234546,-10000010254618,2020,N469


In [35]:
#  merge with ICD data
diagnosis = pd.merge(diagnosis, LU_ICD, on='ICD_DIAG_CD', how='left')
num_diagnosis = diagnosis.groupby(['BENE_ID', 'YR']).size().reset_index(name='NUM_DIAG')
num_diagnosis.head(5)

,BENE_ID,YR,NUM_DIAG
0,-10000010254618,2015,13
1,-10000010254618,2016,14
2,-10000010254618,2017,15
3,-10000010254618,2018,13
4,-10000010254618,2019,18


In [36]:
outpatient_encounters = outpatient[['BENE_ID', 'CLM_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'YR', #'LOS',
                                  'PRNCPAL_DGNS_CD', 'PTNT_DSCHRG_STUS_CD', 'CLM_TOT_CHRG_AMT']]

# perform left joins
outpatient_encounters = outpatient_encounters.merge(num_diagnosis, on=['BENE_ID', 'YR'], how='left')
outpatient_encounters = outpatient_encounters.merge(LU_ICD, left_on='PRNCPAL_DGNS_CD', right_on='ICD_DIAG_CD',
                                                  how='left').drop(columns=['ICD_DIAG_CD'])
outpatient_encounters

,BENE_ID,CLM_ID,CLM_FROM_DT,CLM_THRU_DT,YR,PRNCPAL_DGNS_CD,PTNT_DSCHRG_STUS_CD,CLM_TOT_CHRG_AMT,NUM_DIAG,ICD_Description
0,-10000010254618,-10000930037832,2015-03-16,2015-03-16,2015,R4689,1,10224.89,13,Other symptoms and signs involving appearance ...
1,-10000010254618,-10000930037833,2015-04-27,2015-04-27,2015,R4689,1,11936.39,13,Other symptoms and signs involving appearance ...
2,-10000010254618,-10000930037834,2015-05-23,2015-05-23,2015,R4689,1,8674.15,13,Other symptoms and signs involving appearance ...
3,-10000010254618,-10000930037835,2015-06-01,2015-06-01,2015,R4689,1,1171.57,13,Other symptoms and signs involving appearance ...
4,-10000010254618,-10000930037836,2015-06-14,2015-06-14,2015,R4689,1,11327.22,13,Other symptoms and signs involving appearance ...
...,...,...,...,...,...,...,...,...,...,...
402648,-10000010288007,-10000931486350,2020-11-05,2020-11-05,2020,Z940,1,326.65,21,Kidney transplant status
402649,-10000010288007,-10000931486351,2021-05-11,2021-05-11,2021,Z940,1,272.70,17,Kidney transplant status
402650,-10000010288007,-10000931486352,2021-06-08,2021-06-08,2021,Z940,1,272.70,17,Kidney transplant status
402651,-10000010288007,-10000931486353,2022-06-11,2022-06-11,2022,S5290X,1,82.02,18,"A Unspecified fracture of unspecified forearm,..."


In [37]:
outpatient_encounters.to_csv('/content/drive/MyDrive/Data/Output/outpatient_encounters.csv', index=False)
diagnosis.to_csv('/content/drive/MyDrive/Data/Output/outpatient_diagnosis.csv', index=False)